# Guardar el modelo

In [ ]:
import pandas as pd

df_encoded=pd.read_csv('https://raw.githubusercontent.com/Cl01010608/En2_PYA/main/Data_Codificada.csv', encoding='latin-1', sep=',' )

In [ ]:
print(df_encoded)

        Arma_Empleada  Departamento  Ciudad       Fecha  Cantidad  \
0                   0             0       0  2022-07-09  0.007366   
1                   0             0       0  2022-11-04  0.007366   
2                   0             0      25  2022-03-15  0.007366   
3                   0             0      25  2022-12-02  0.007366   
4                   0             0      26  2022-04-06  0.007366   
...               ...           ...     ...         ...       ...   
142873              7            19     831  2022-07-21  0.036555   
142874              7            19     831  2022-08-26  0.036555   
142875              7            19     843  2022-01-27  0.036555   
142876              5            19     271  2022-08-28  0.036555   
142877              1            19     346  2022-02-22  0.036555   

        Tipo_de_robo  CAI_Departamentos  Mes   Ano  Cantidad_unitaria  \
0                  4              640.0    7  2022                1.0   
1                  4     

In [ ]:
new_data=df_encoded.drop('Fecha', axis=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
variables_numericas = new_data.select_dtypes(include=['float64', 'int64'])
# Define X e y
X = variables_numericas.drop(columns=['Cantidad','Arma_Empleada','Ciudad','Tipo_de_robo','Cantidad_unitaria'])
y = variables_numericas['Cantidad']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define los modelos con los parámetros a ajustar
models = {
    'Linear Regression': (LinearRegression(), {}),
    'Ridge Regression': (Ridge(), {'regressor__alpha': [0.1, 1.0, 10.0],'regressor__max_iter': [100, 1000]}),
    'Lasso Regression': (Lasso(), {'regressor__alpha': [0.1, 1.0, 10.0],'regressor__max_iter': [100, 1000]}),
    'Random Forest': (RandomForestRegressor(), {'regressor__n_estimators': [100, 200, 300],'regressor__criterion': ['squared_error'], 'regressor__max_samples': [1/3, 2/3]})
}
# Crear una lista para almacenar los mejores estimadores
pipelines = []
mse_values = []
mae_values = []
mse_iter_values = {model_name: [] for model_name in models}
mae_iter_values = {model_name: [] for model_name in models}

for model_name, (model, params) in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])
    # Ajustar el pipeline con GridSearchCV para optimizar hiperparámetros
    grid_search = GridSearchCV(pipeline, params, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    pipelines.append((model_name, best_estimator))
    # Evaluar los modelos en el conjunto de prueba y calcular MSE y MAE
    y_pred = best_estimator.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse_values.append(mse)
    mae_values.append(mae)

    print(f"{model_name}:")
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    # Almacenar los errores MSE y MAE para cada iteración
    for i in range(10):
        grid_search = GridSearchCV(pipeline, params, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_estimator = grid_search.best_estimator_

        # Evaluar el modelo en el conjunto de prueba y calcular MSE y MAE
        y_pred = best_estimator.predict(X_test)
        mse_iter = mean_squared_error(y_test, y_pred)
        mae_iter = mean_absolute_error(y_test, y_pred)
        mse_iter_values[model_name].append(mse_iter)
        mae_iter_values[model_name].append(mae_iter)

# Seleccionar el modelo con el menor MSE
best_model_index = min(enumerate(mse_values), key=lambda x: x[1])[0]
best_model_name = list(models.keys())[best_model_index]
best_mse = mse_values[best_model_index]
print(f"Best model based on test MSE: {best_model_name}")
print(f"Mean Squared Error: {best_mse}")

# Obtener los parámetros del mejor modelo
print(f"Parameters of the best model ({best_model_name}):")
best_params = pipelines[best_model_index][1].named_steps['regressor']
print(best_params.get_params())


Linear Regression:
Mean Squared Error: 0.022150422748984547
Mean Absolute Error: 0.049772439458523256
Ridge Regression:
Mean Squared Error: 0.022150438127728673
Mean Absolute Error: 0.04977242009636104
Lasso Regression:
Mean Squared Error: 0.02292446275920496
Mean Absolute Error: 0.05186170824193703
Random Forest:
Mean Squared Error: 0.021959653518504693
Mean Absolute Error: 0.047909433590405635


KeyboardInterrupt: 

In [ ]:
import joblib

# Save the best model
best_model = pipelines[best_model_index][1]
joblib.dump(best_model, 'best_model_SqEr.pkl')

['best_model_SqEr.pkl']

# Generar API

In [3]:
import joblib

# Cargar el modelo
model = joblib.load('best_model_SqEr.pkl')

# Ejemplo de datos para la predicción
example_data = [12,  136, 4, 2022, 13575692, 2]

# Realizar la predicción
prediction = model.predict([example_data])

# Imprimir el resultado
print(prediction)

[0.10655173]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)
# Cargar el modelo
model = joblib.load('best_model_SqEr.pkl')

# Ruta para la API
@app.route('/api/prediction', methods=['POST'])
def predict():
    # Obtener los datos de la petición HTTP
    data = request.get_json()

    # Extraer los valores de las variables
    departamento = data['Departamento']
    cai_departamentos = data['CAI_Departamentos']
    mes = data['Mes']
    ano = data['Ano']
    poblacion = data['Poblacion']
    pobreza_monetaria = data['Pobreza_Monetaria']

    # Realizar la predicción utilizando el modelo cargado
    prediction = model.predict([[departamento, cai_departamentos, mes, ano, poblacion, pobreza_monetaria]])

    # Devolver la predicción como respuesta
    response = {'prediction': prediction[0]}
    return jsonify(response)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
